In [1]:
import pandas as pd
import tabula
import os
from PyPDF2 import PdfReader
import subprocess

In [1861]:
# get all files in a folder
folder = './ceap_2022'
file_names = os.listdir(folder)
len(file_names)

506

### Functions

In [3]:
# funcao que retira a palavra vereador do nome e retorna primeiro
# nome e sobrenome
def _shear_names( str_name ):
    
    #strip word vereador: and split first name and last names
    names = str_name.split(" ", 1)[1]
    names = names.split(" ", 1)
    
    return names

def get_specific_line(string, line_number):
    """
    This function returns the specified line from a string separated by new lines.
    If the line number is out of range, it returns None.
    """
    lines = string.split('\n')
    
    if 0 <= line_number < len(lines):
        return lines[line_number]
    else:
        return None
    
def get_name(index, line):
    """
    This function extracts the name of the deputy from the PDF file.
    """
    with open(folder+'/'+file_names[index], 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return get_specific_line(text, line).split(':')[1]

def open_pdf(index):
    """
    This function opens the PDF file in the default PDF viewer.
    """
    subprocess.Popen(['xdg-open', folder+'/'+file_names[index]])

### Opens a pdf file from the list

In [ ]:
index = 0
#index 12 tem pdf em branco
try:
    dfs = tabula.read_pdf( folder+'/'+file_names[index], pages='all', pandas_options={'dtype':'str'}) # entrega um Dataframe
    #print(dfs[0])
except Exception:
    print("File doesn't exist")
    dfs = None
open_pdf(index)
dfs[0]

Got stderr: fev 25, 2025 12:26:13 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
fev 25, 2025 12:26:13 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$),"19.803,67",(%)
0,11.220.341/0001-99,19/05/2021,000.001.202,BATARA COMERCIO DE,COMBUSTIVEL,"4.500,00","15.303,67","27,27"
1,04.819.323/0001-62,20/05/2021,401-E/2021,KAELE LTDA.,LOC.VEICULO TERRESTRE FLUVIAL,"9.000,00","6.303,67","54,55"
2,04.867.888/0001-15,20/05/2021,811,PLUTÃO DA AMAZÔNIA LTDA,DIVUL. ATIVIDADE PARLAMENTAR,"3.000,00","3.303,67","18,18"
3,NaN,NaN,NaN,NaN,TOTAL:,"16.500,00","3.303,67","100,00"


Opening in existing browser session.


### Format Data Frame

In [ ]:
# option #1
for df in dfs:
    # armazenar o nome do vereador
    _firstcol = dfs[0].columns[0]
    names = _shear_names( primary_df[_firstcol][1] )
    
    # procurar a linha que se encontra o titulo CNPJ
   
    new_frame = df.drop( range(0,9) )
    new_frame.insert(0, 'fname', names[0])
    new_frame.insert(1, 'lname', names[1])
    new_frame.insert(4, 'Nro.Documento', None)
    new_frame   =   new_frame.head( -1 )
    new_frame   =   new_frame.iloc[:, :-2]
    new_frame.rename(columns={_firstcol:'CNPJ', 'Unnamed: 0':'Data Documento', 'Unnamed: 1':'Fornecedor', 'Unnamed: 2':'Item de Despesa', 'Unnamed: 3':'Valor (R$)'}, inplace=True)

    # Split 'Data Documento' and assign the second part to 'Nro.Documento'
    new_frame['Nro.Documento'] = new_frame['Data Documento'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else None)
    new_frame['Data Documento'] = new_frame['Data Documento'].apply(lambda x: x.split()[0])

    print( new_frame.head())

### Format Data Frame (simple version)

In [1849]:
for df in dfs:
    # armazebnar o nome do vereador
    name = get_name(index, 3)

    df.drop( df.columns[[6,7]], axis=1, inplace=True)
    df.insert(0, 'Nome', name)
    df.drop( df.index[-1], inplace=True)

dfs[0]

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
0,CICERO CUSTODIO DA SILVA,11.220.341/0001-99,14/12/2021,000001354,BATARA COMERCIO DE,COMBUSTIVEL,"7.800,00"
1,CICERO CUSTODIO DA SILVA,04.867.888/0001-15,14/12/2021,884,PLUTÃO DA AMAZÔNIA LTDA,DIVUL. ATIVIDADE PARLAMENTAR,"3.200,00"
2,CICERO CUSTODIO DA SILVA,04.819.323/0001-62,15/12/2021,401-L/2021,KAELE LTDA.,LOC.VEICULO TERRESTRE FLUVIAL,"9.000,00"


### Append to dfs

In [1850]:
for df in dfs:
    MasterDF =  pd.concat( [MasterDF, df], ignore_index=True)

### Create Master DF based in the fisrt DataFrame

In [10]:
MasterDF = pd.DataFrame()

for df in dfs:
    MasterDF = pd.concat([MasterDF, df], ignore_index=True)


In [603]:
backup = MasterDF.copy() # last 93

In [588]:
MasterDF = backup.copy()

### Visualization

In [ ]:
#Just visizualize Master DaytaFrame
MasterDF.tail()

,Nome,CNPJ,Data Documento,Nro.Documento,Fornecedor,Item de Despesa,Valor (R$)
1568,ROSIVALDO OLIVEIRA CORDOVIL,02.558.157/0001-62,03/05/2021,1237891187739,TELEFONICA BRASIL S.A.,TELEFONIA MOVEL,"629,99"
1569,ROSIVALDO OLIVEIRA CORDOVIL,23.029.382/0001-81,20/05/2021,6266,AUTO POSTO UNIÃO VII LTDA,COMBUSTIVEL,"4.850,00"
1570,ROSIVALDO OLIVEIRA CORDOVIL,05.526.766/0001-28,20/05/2021,1672,GRAFICA E EDITORA VITORIA,DIVUL. ATIVIDADE PARLAMENTAR,"4.020,05"
1571,ROSIVALDO OLIVEIRA CORDOVIL,19.713.467/0001-07,20/05/2021,A10,SUPORTE LOCADORA DE,LOC.VEICULO TERRESTRE FLUVIAL,"8.700,00"
1572,CICERO CUSTODIO DA SILVA,11.220.341/0001-99,19/05/2021,000.001.202,BATARA COMERCIO DE,COMBUSTIVEL,"4.500,00"
1573,CICERO CUSTODIO DA SILVA,04.819.323/0001-62,20/05/2021,401-E/2021,KAELE LTDA.,LOC.VEICULO TERRESTRE FLUVIAL,"9.000,00"
1574,CICERO CUSTODIO DA SILVA,04.867.888/0001-15,20/05/2021,811,PLUTÃO DA AMAZÔNIA LTDA,DIVUL. ATIVIDADE PARLAMENTAR,"3.000,00"
1575,CICERO CUSTODIO DA SILVA,11.220.341/0001-99,14/12/2021,000001354,BATARA COMERCIO DE,COMBUSTIVEL,"7.800,00"
1576,CICERO CUSTODIO DA SILVA,04.867.888/0001-15,14/12/2021,884,PLUTÃO DA AMAZÔNIA LTDA,DIVUL. ATIVIDADE PARLAMENTAR,"3.200,00"
1577,CICERO CUSTODIO DA SILVA,04.819.323/0001-62,15/12/2021,401-L/2021,KAELE LTDA.,LOC.VEICULO TERRESTRE FLUVIAL,"9.000,00"


### Manual corrections

In [504]:
MasterDF.drop( MasterDF.index[-1], inplace=True) 

In [596]:
MasterDF.drop(MasterDF.columns[[7,8]], axis=1, inplace=True)

In [1830]:
MasterDF.loc[len(MasterDF)] = ['DIONE DE CARVALHO DOS SANTOS', '04.819.323/0001-62', '22/02/2021', '201-B/2021', 'KAELE LTDA.', 'LOC.VEICULO TERRESTRE FLUVIAL', '9.000,00']

### Save

In [1860]:
# Save DataFrame to a Pickle file
MasterDF.to_pickle(folder+'.pkl')

### Load

In [ ]:
# Load DataFrame from a Pickle file
MasterDF = pd.read_pickle(folder+'.pkl')

### Convert to CSV

In [1859]:
MasterDF.to_csv(folder+'.csv', index=False, sep=';')